In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with h5py.File("/kaggle/input/cat-images-dataset/train_catvnoncat.h5", 'r') as f:
    print(f.keys())
with h5py.File("/kaggle/input/cat-images-dataset/test_catvnoncat.h5", "r") as f:
    print(f.keys())

In [ ]:
def load_data():
    train_data = h5py.File("/kaggle/input/cat-images-dataset/train_catvnoncat.h5", 'r')
    X_train_original = np.array(train_data["train_set_x"]).astype(np.float64)
    Y_train_original = np.array(train_data['train_set_y'])
    test_data = h5py.File("/kaggle/input/cat-images-dataset/test_catvnoncat.h5", "r")
    X_test_original = np.array(test_data['test_set_x']).astype(np.float64)
    Y_test_original = np.array(test_data['test_set_y'])
    data = {'X_train_original': X_train_original, 'Y_train_original': Y_train_original, 'X_test_original': X_test_original, 'Y_test_original': Y_test_original}
    return data

In [ ]:
data = load_data()

In [ ]:
for name in data:
    print(name, data[name].shape)

In [ ]:
X_train_original = data["X_train_original"]
X_test_original = data["X_test_original"]
Y_train_original = data["Y_train_original"]
Y_test_original = data["Y_test_original"]

X_train_flatten = X_train_original.reshape(X_train_original.shape[0], -1).T
X_test_flatten = X_test_original.reshape(X_test_original.shape[0], -1).T
Y_train = Y_train_original.reshape(1, Y_train_original.shape[0])
Y_test = Y_test_original.reshape(1, Y_test_original.shape[0])
print(X_train_flatten.shape)
print(Y_train.shape)
print(X_test_flatten.shape)
print(Y_test.shape)

In [ ]:
X_train = X_train_flatten/255.0
X_test = X_test_flatten/255.0

In [ ]:
def initialize_params(dimensions):
    params = {}
    L = len(dimensions)
    
    for l in range(1, L):
        params['W' + str(l)] = np.random.randn(dimensions[l], dimensions[l-1])*0.1
        params['b' + str(l)] = np.zeros((dimensions[l], 1))
    return params

In [ ]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A
    

In [ ]:
def relu(Z):
    A = np.maximum(0, Z)
    return A

In [ ]:
def forward_prop(X, params):
    L = len(params)//2
    cache = {}
    A_prev = X
    cache['A0'] = X
    for l in range(1,L):
        curr_W = params['W' + str(l)]
        curr_b = params['b' + str(l)]
        Z = np.dot(curr_W, A_prev) + curr_b
        A_prev = relu(Z)
        cache['Z' + str(l)] = Z
        cache['A' + str(l)] = A_prev

    cache['Z' + str(L)] = np.dot(params['W' + str(L)], A_prev) + params['b' + str(L)]
    cache['A' + str(L)] = sigmoid(cache['Z' + str(L)])

    return cache

In [ ]:
def cost_fn(Y, A):
    m = Y.shape[1]
    cost = 1/m*np.sum(-(Y*np.log(A) + (1-Y)*np.log(1-A)))
    return cost

In [ ]:
def relu_derivative(Z):
    derivative = (Z > 0).astype(int)
    return derivative

In [ ]:
def back_prop(Y, params, cache):
    L = len(cache)//2
    m = Y.shape[1]
    A_Last = cache['A' + str(L)]
    grads = {}
    dZ = A_Last - Y
    grads['dZ' + str(L)] = dZ
    grads['dW' + str(L)] = 1/m*np.dot(dZ, cache['A' + str(L-1)].T)
    grads['db' + str(L)] = 1/m*np.sum(dZ, axis = 1, keepdims = True)

    for l in range (L-1, 0, -1):
        grads['dZ' + str(l)] = np.dot(params['W' + str(l+1)].T, grads['dZ' + str(l+1)])*relu_derivative(cache['Z' + str(l)])
        grads['dW' + str(l)] = 1/m*np.dot(grads['dZ' + str(l)], cache['A' + str(l-1)].T)
        grads['db' + str(l)] = 1/m*np.sum(grads['dZ' + str(l)], axis=1, keepdims=True)

    return grads
    

In [ ]:
def update_params(params, grads, learning_rate):
    L = len(params)//2
    for l in range (1,L):
        params['W' + str(l)] -= learning_rate*grads['dW' + str(l)]
        params['b' + str(l)] -= learning_rate*grads['db' + str(l)]
    return params

In [ ]:
def model (X, Y, dimensions, learning_rate, num_iterations):
    params = initialize_params(dimensions)
    L = len(dimensions)-1
    costs = []
    for i in range(num_iterations):
        cache = forward_prop(X, params)
        
        A = cache['A' + str(L)]
        cost = cost_fn(Y, A)
        costs.append(cost)
        grads = back_prop(Y, params, cache)

        params = update_params(params, grads, learning_rate)

        if (i%1000==0):
            print("Cost after " , (i+1), " iterations: ", cost)
    return params, costs

In [ ]:
dimensions = [X_train.shape[0], 5, 4, 1]
params, costs = model(X_train, Y_train, dimensions, 0.05, 10000)


In [ ]:
def plot_costs(costs):
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.show()

In [ ]:
plot_costs(costs)

In [ ]:
def predict(X, Y, params):
    L = len(params)//2
    cache = forward_prop(X, params)
    A = cache['A' + str(L)]
    Y_pred = (A > 0.5).astype(int)
    accuracy = np.mean(Y_pred == Y)
    print("accuracy = ", accuracy)
    print(A[:10])
    print(Y[:10])

In [ ]:
predict(X_test, Y_test, params)